In [3]:
%load_ext autoreload
%autoreload 2
from recsys.model import MultitaskRecommender
from recsys.dataset import NewsDataModule

model = MultitaskRecommender(768)
if "datamodule" not in locals():
    datamodule = NewsDataModule("../data", batch_size=32)
    datamodule.prepare_data()
    datamodule.setup()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
datamodule.train_dataset[0:5]
model.on_validation_start()
model.on_validation_epoch_start()
model.validation_step(datamodule.train_dataset[0:5], 0)
model.on_validation_epoch_end()

/Users/Matey/project/MTRec-RecSys/.venv/lib/python3.11/site-packages/pytorch_lightning/core/module.py:436: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
/Users/Matey/project/MTRec-RecSys/.venv/lib/python3.11/site-packages/pytorch_lightning/core/module.py:436: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


In [4]:
from ebrec.utils._constants import (
    DEFAULT_ARTICLE_ID_COL,
    DEFAULT_BODY_COL,
    DEFAULT_CATEGORY_STR_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_TOPICS_COL,
    DEFAULT_USER_COL,
)

self = datamodule.train_dataset
batch = self.data[0:5]
history_input = self.lookup_matrix[batch[DEFAULT_HISTORY_ARTICLE_ID_COL].to_list()]
candidate_input = self.lookup_matrix[batch[DEFAULT_INVIEW_ARTICLES_COL].to_list()]


In [7]:
candidate_input.shape

(5, 5, 1, 768)

AttributeError: 'PolarsDataFrameWrapper' object has no attribute 'dataframe'

In [50]:
scores = model.predictions[0]
labels = model.labels[0]
from torch.nn import functional as F

F.binary_cross_entropy_with_logits(model.predictions[0], model.labels[0].float())

tensor(0.7424)

In [95]:
print(scores.shape) # (batch_size, candidates)
print(labels.shape) # (batch_size, candidates)

# loss = F.cross_entropy(scores, labels)
# print(loss)

torch.Size([5, 5])
torch.Size([5, 5])


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [56]:
model.labels[0].T

tensor([[1, 0, 0, 0, 0],
        [0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0]])